# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [38]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [39]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [40]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [19]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Task1 : Getting the artist, song title and song length where the sessionID = 388 and itemInSession = 4 

##### For this table we'll use session_id, item_in_session as composite key for getting unique records for item in session and for filtering inaou WHERE clause

In [20]:
# Creating the table 

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + """(session_id int, 
                    item_in_session int, 
                    artist_name text, 
                    song_title text, 
                    song_length float, 
                    PRIMARY KEY(session_id, item_in_session))"""
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [31]:
# Opening and reading the file and inserting the data

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_session (session_id, item_in_session, artist_name, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})

In [34]:
# Running the select to validate results

query = "select artist_name, song_title, song_length from song_playlist_session where session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

('Unable to complete the operation against any hosts', {<Host: 127.0.0.1 datacenter1>: ConnectionException('Pool is shutdown',)})


#### Task2 : Getting the artist, song title , user first and last name where the userid = 10, sessionid = 182

##### For this table we'll use session_id and user_id as composite key for getting unique records and item_in_session as clustering key for ordering the results

In [23]:
# Creating table

query = "CREATE TABLE IF NOT EXISTS song_playlist_user_session "
query = query + """(user_id int, 
                    item_in_session int,
                    session_id int,
                    artist_name text, 
                    song_title text, 
                    first_name text, 
                    last_name text,
                    PRIMARY KEY ((session_id, user_id), item_in_session ))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Opening and reading the file and inserting the datafile = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_playlist_user_session (user_id, item_in_session, session_id,  artist_name, song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (int(line[10]), int(line[3]), int(line[8]), line[0], line[9], line[1], line[4]))


# Running the select to validate results

query = "select artist_name, song_title, first_name, last_name from song_playlist_user_session where user_id = 10 and session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)
                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Task3 : Getting the user first and last name who listened to the song 'All Hands Against His Own'

##### For this table we'll use song_title and user_id as composite key for getting unique records and being able to filter the result by the song title

In [27]:
# Creating table

query = "CREATE TABLE IF NOT EXISTS user_list "
query = query + """(song_title text, 
                    user_id int,
                    first_name text, 
                    last_name text,
                    PRIMARY KEY (song_title, user_id))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

    
# Opening and reading the file and inserting the data


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_list (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# Running the select to validate results

query = "select first_name, last_name from user_list where song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)
                    

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Droping tables

In [28]:
query = "DROP TABLE IF EXISTS song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS song_playlist_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS user_list"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()